In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

all_users = ff_utils.search_metadata('/search/?type=user', my_auth)
print(len(all_users))

f = open('/Users/koray/downloads/hms-dbmi.json')
dicto = json.loads(f.read())
print(len(dicto))


532
11513


In [17]:
dicto = [i for i in dicto if i.get('Email')]
print(len(dicto))
all_data = []
all_cs = []
found = 0
nfound = 0
for i in all_users:
    em = i['email']
    
    match = False
    cs = []
    count = 0
    for x in dicto:
        em2 = x['Email']
        c = x['Connection']
        if em.lower().strip() == em2.lower().strip():
            #print(em2, c)
            match = True
            cs.append(c)
            log = x['Logins Count']
            count += log
    cs = sorted(cs)
    
    cs = ' | '.join(cs)
    all_cs.append(cs)
    

    if match:
        found += 1
        all_data.append({'mail':em, 'count':count, 'services': cs})
    else:
        nfound += 1
print(found, nfound)

11328
151 381


In [15]:
from collections import Counter
print(Counter(all_cs))

Counter({'': 381, 'google-oauth2': 122, 'github | google-oauth2': 18, 'github': 8, 'Username-Password-Authentication | github | google-oauth2': 1, 'google-oauth2 | hms-it': 1, 'Username-Password-Authentication | google-oauth2': 1})


In [21]:
all_data = sorted(all_data, key=lambda k:[k['count']], reverse=True)

In [27]:
for i in all_data:
    print("{:<35} {}".format(i['mail'], i['count']))

4dndcic@gmail.com                   41650
chad_serrant@hms.harvard.edu        1320
burak_alver@hms.harvard.edu         565
carl_vitzthum@hms.harvard.edu       506
sarah_reiff@hms.harvard.edu         461
andrew_schroeder@hms.harvard.edu    391
luisa_mercado@hms.harvard.edu       213
alexander_balashov@hms.harvard.edu  210
koray_kirli@hms.harvard.edu         182
yuchuanw@andrew.cmu.edu             82
hu_jin@hms.harvard.edu              73
ankita.nand@umassmed.edu            71
wangxiaotao686@gmail.com            60
burakhan@gmail.com                  53
duplexa@gmail.com                   53
yanzm@eng.ucsd.edu                  51
akukalev@gmail.com                  41
hakan.ozadam@umassmed.edu           40
ayildiri@usc.edu                    39
johnsonjp@gmail.com                 38
gbonora@uw.edu                      36
pzhao@bio.fsu.edu                   35
dhawal.sjain@gmail.com              35
polles@usc.edu                      33
sergpolly@gmail.com                 32
foxjuly@gmail

In [32]:
def write_outfile(terms, filename, pretty=False):
    '''terms is a list of dicts
        write to file by default as a json list or if pretty
        then same with indents and newlines
    '''
    with open(filename, 'w') as outfile:
        if pretty:
            json.dump(terms, outfile, indent=4)
        else:
            json.dump(terms, outfile)
            
            
            
            
            
listem = [{"a": "araba", "b": "badana"}, {"a": "adana", "b": "balaba"}]

write_outfile(listem, 'bir.json', pretty=False)
write_outfile(listem, 'birim.json', pretty=True)


In [56]:
opf_set = ('search/?type=ExperimentSet&other_processed_files.title%21=No+value&field=status'
               '&field=other_processed_files&field=experiments_in_set.other_processed_files')
opf_exp = ('search/?type=ExperimentSet&other_processed_files.title=No+value'
           '&experiments_in_set.other_processed_files.title%21=No+value'
           '&field=experiments_in_set.other_processed_files&field=status')
    
res_set = ff_utils.search_metadata(opf_set, my_auth)
res_exp = ff_utils.search_metadata(opf_exp, my_auth)
print(len(res_set))
print(len(res_exp))

89
49


In [72]:
import time


from dcicutils import ff_utils
from dcicutils import es_utils
files = []

all_sets = res_set + res_exp

for a_set in all_sets:
    if a_set.get('other_processed_files'):
        for case in a_set['other_processed_files']:
            files.extend([i['uuid'] for i in case['files']])
    if a_set.get('experiments_in_set'):
        for an_exp in a_set['experiments_in_set']:
            for case in an_exp['other_processed_files']:
                files.extend([i['uuid'] for i in case['files']])
print(len(files))

for chunk in [100, 500, 1000, 2000, 5000]:
    time1 = time.time()
    resp =  ff_utils.get_es_metadata(files, chunk_size=chunk, key=my_auth)
    time2 = time.time()
    print(round((time2-time1), 1), 'sec for collection', chunk)

3617
16.4 sec for collection 100
15.6 sec for collection 500
13.4 sec for collection 1000


GET http://search-fourfront-webprod2-fkav4x4wjvhgejtcg6ilrmczpe.us-east-1.es.amazonaws.com:80/_all/_search?size=2000&from=0 [status:400 request:2.445s]


RequestError: TransportError(400, 'search_phase_execution_exception', 'failed to create query: {\n  "bool" : {\n    "must" : [\n      {\n        "terms" : {\n          "_id" : [\n            "b1956b2f-f111-4678-a180-1906ec732358",\n            "1a610d44-4811-40ba-a4ab-8391d73c1950",\n            "111dfd62-1a3c-4722-9697-7f6031967149",\n            "af6e1f5a-9bbc-4216-b195-5c5410fbe92e",\n            "0398d788-13da-4d54-9505-a929891651af",\n            "ceb4892f-ccd1-419a-a913-26185cf05b81",\n            "5819a3df-c2e8-4313-a66a-9051d51ffaba",\n            "28d0c9cf-73bc-4918-b181-45678151e6f2",\n            "a1ff380f-0870-4844-95bd-fd9497ea9644",\n            "174149bb-8115-4cd9-bcfc-810d98e31f0c",\n            "72418029-04d3-40f5-aa17-cb548a1b5d41",\n            "bc8492c4-91be-4d97-bf68-dbd5497250e1",\n            "50b1dce8-85a3-424e-8d02-359802e4e054",\n            "c8c153ed-eeaf-45fb-9bd6-8a8b7f454da3",\n            "bfffae01-277f-4418-b5ae-8ca5ef9e566c",\n            "6923150f-fcd6

In [71]:
import time
time1 = time.time()

files = 'search/?type=FileProcessed&field=status'
file_results = ff_utils.search_metadata(files + '&experiment_sets.uuid!=No+value', ff_env='data')
file_results += ff_utils.search_metadata(files + '&experiments.uuid!=No+value', ff_env='data')
file_results += ff_utils.search_metadata(files + '&status=deleted&status=replaced&experiment_sets.uuid!=No+value',
                                         ff_env='data')
file_results += ff_utils.search_metadata(files + '&status=deleted&status=replaced&experiments.uuid!=No+value',
                                         ff_env='data')
print(len(file_results))

time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

5026
20.1 sec for collection
